In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("progress_afp.csv")

In [12]:
df = df[['data', "day_fixed"]]
df['day_fixed'] = list(map(pd.Timestamp, df['day_fixed'].tolist()))

In [13]:
df.head(3)

,data,day_fixed
0,シリア 南部 の 避難 民 27 万 人 超える 政府 軍 攻勢 で 急増 国連 発表 内戦...,2018-07-03 07:06:00
1,メキシコ 次期 大統領 、 トランプ 氏 と 初 の 電話 会談 「 素晴らしい 」 滑り出...,2018-07-03 10:55:00
2,比 亜 迪 の 電池 工場 、 青海 で 操業 開始 世界 最大 規模 中国 の 電気 自動...,2018-07-03 09:22:00


In [9]:
import numpy as np
from functools import partial
from dateutil import parser
import datetime

def diff_date(t1, t2):
    if t1 < t2:
        tmp = t1
        t1 = t2
        t2 = tmp
    return (t1-t2).to_timedelta64().astype("timedelta64[h]")/np.timedelta64(1,'h')

def within(t1, base, hour=12):
    return diff_date(t1, base) <= hour

def get_around_12(df, base, hour=12, column="day_fixed"):
    func = partial(within, base=base, hour=hour)
    return df[list(map(func, df[column].tolist()))]

def get_day(start, plus):
    return pd.Timestamp(start + datetime.timedelta(days=plus))

In [14]:
start = datetime.datetime(2017,2,1,12,0,0)
day = get_day(start, 0)
targets = get_around_12(df, day, hour=12*30)

In [16]:
targets.head(3)

,data,day_fixed
10927,ロシア は 「 大統領 選 に 干渉 する な 」 仏 外相 が 警告 フランス の ジャン...,2017-02-16 09:59:00
10930,NZ クライストチャーチ で 森林 火災 、 非常 事態 を 宣言 ニュージーランド 南島 ...,2017-02-16 12:42:00
10931,動画 配信 中 に 銃撃 、 幼児 ら 2 人 死亡 米 シカゴ 米 イリノイ （ Ilin...,2017-02-16 08:43:00


In [17]:
import tensorflow as tf
import tensorflow_hub as hub

class Word2VecVectorizer:
    def __init__(self):
        pass

    def fit(self, X, y):
        return self

    def transform(self, X):
        if isinstance(X, pd.core.series.Series):
            X = X.tolist()        
        return self.texts_encoder(X)

    def texts_encoder(self, texts):
        with tf.Graph().as_default():
            embed = hub.Module("https://tfhub.dev/google/nnlm-ja-dim128/1")
            embeddings = embed(texts)
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                sess.run(tf.tables_initializer())
                result = sess.run(embeddings)
        return result

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
word_vectors = Word2VecVectorizer().transform(targets['data'])

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(word_vectors)

In [20]:
import markov_clustering as mc
result = mc.run_mcl(similarities)
clusters = mc.get_clusters(result)

In [30]:
for i, cluster in enumerate(clusters):
    print("[{}]".format(i))
    for c in cluster:
        print(targets.iloc[c]['data'].replace("\n", ""))
    print("")

1

In [31]:
similarities

array([[1.        , 0.5112249 , 0.47024447, ..., 0.66792464, 0.67268664,
        0.39303657],
       [0.5112249 , 1.        , 0.6203477 , ..., 0.5470191 , 0.6137175 ,
        0.20962489],
       [0.47024447, 0.6203477 , 1.        , ..., 0.63104254, 0.47594666,
        0.41406703],
       ...,
       [0.66792464, 0.5470191 , 0.63104254, ..., 1.        , 0.6481949 ,
        0.50885403],
       [0.67268664, 0.6137175 , 0.47594666, ..., 0.6481949 , 1.0000002 ,
        0.5160222 ],
       [0.39303657, 0.20962489, 0.41406703, ..., 0.50885403, 0.5160222 ,
        0.9999999 ]], dtype=float32)